In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
import sys as sys
import time
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import BasicTools as BT
import WaveTools as WT
import PlotTools as PT
import FFTTools as FFTT

In [2]:
nh = 2
h = 1. / nh
xCellFiller, wavesFiller = WT.MakeWaves(nh, h)
xCell = xCellFiller[0:nh - 1]
waves = wavesFiller[0:nh - 1, :]
nh = 2 * nh
print(xCell)
print(waves)
# xNode, wavesNode = WT.MakeNodeWaves(nh, h)
xBound = [[0, 2], [2, 4], [4, 6], [8, 12]] # [[0, 1], [0, 1, 2], [0, 1], [0, 1]] 
print(len(xBound[0]))# == [2, 2, 2, 2])

[0.25]
[[1.         0.63661977]]
2


In [3]:
def MakeAMRWaves(nh_max, xBound):
    x, y = BT.MakeXY(nh_max, xBounds = xBound)
    n = len(x) - 1
    xCell = 0.5 * (x[0:n] + x[1:n + 1])
    hDiag = x[1:n + 1] - x[0:n]
    waves = WT.CellWaves(nh, x)
    hs = np.zeros((n, n), float)
    np.fill_diagonal(hs, hDiag)
    # hs[0, 0] = 1.
    zeroMat = np.zeros((n, nh_max - n), float)
    hMat = LA.inv(hs) # np.append(hs, zeroMat).reshape(nh_max, n)
#     hMat[0, 0] = 1.
    print(hMat)
    print('')
    print(waves)
    waves = hMat @ waves_all
    
    print('')
    print(waves)
    return xCell, waves

In [4]:
def MakeAMRNodeWaves(nh_max, xBound):
    x, y = BT.MakeXY(nh_max, xBounds = xBound)
    n = len(x) - 1
    x = x[0:n]
    print(x)
    waves = WT.NodeWaves(nh_max, x, n)
    return x, waves

In [5]:
nh = 16
h = 1 / 16.
xCell, waves = MakeAMRWaves(nh, xBound)
xNode, wavesNode = MakeAMRNodeWaves(nh, xBound)
PT.PlotWaves(nh, h, waves, xNode, wavesNode, xBound = xBound, plotNode = True)
print(waves)

[[ 2.  0.  0.  0.  0.  0.]
 [ 0. 16.  0.  0.  0.  0.]
 [ 0.  0. 16.  0.  0.  0.]
 [-0. -0. -0. 16. -0. -0.]
 [ 0.  0.  0.  0. 16.  0.]
 [ 0.  0.  0.  0.  0.  4.]]

[[ 1.00000000e+00  3.18309886e-01  1.94908592e-17  0.00000000e+00
  -1.94908592e-17  1.06103295e-01  1.94908592e-17  0.00000000e+00
  -1.94908592e-17  6.36619772e-02  1.94908592e-17  0.00000000e+00
  -1.94908592e-17  4.54728409e-02  1.94908592e-17  0.00000000e+00]
 [ 1.00000000e+00 -1.21149487e-02 -6.09059599e-02  2.33077018e-02
   5.62697698e-02 -3.27496611e-02 -4.90133315e-02  3.97887358e-02
   3.97887358e-02 -4.40121806e-02 -2.94079989e-02  4.52824138e-02
   1.87565899e-02 -4.37421339e-02 -8.70085141e-03  3.97887358e-02]
 [ 1.00000000e+00 -3.45004549e-02 -5.16335796e-02  5.62697698e-02
   2.33077018e-02 -5.78151665e-02  1.15001516e-02  3.97887358e-02
  -3.97887358e-02 -1.03267159e-02  5.19159068e-02 -1.87565899e-02
  -4.52824138e-02  3.70827906e-02  2.47779285e-02 -3.97887358e-02]
 [ 1.00000000e+00 -5.16335796e-02 -3.4500

NameError: name 'waves_all' is not defined

In [ ]:
blah = np.eye(5)
bluh = np.zeros((5, 3), float)
print(bluh)
print(np.append(blah, bluh).reshape(8, 5).T)

In [ ]:
M = np.arange(16).reshape(4, 4)
O = M
O.T[::4] = 0
print(O)